<a href="https://colab.research.google.com/github/AksenovEA/Test/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import re

print(" Загрузка данных...")

df = pd.read_csv('datasets.csv', encoding='utf-8')

print(f" Загружено {len(df)} отзывов")
print(f"Колонки: {df.columns.tolist()}")
print(f"\n Первые 5 строк:")
print(df.head())

df = df.rename(columns={'sentiment': 'rating'})

print(f"\n Преобразование рейтингов...")
print(f"Уникальные значения оценок: {sorted(df['rating'].unique())}")

print("\n Примеры отзывов по оценкам:")
for rating in sorted(df['rating'].unique()):
    samples = df[df['rating'] == rating].head(2)
    if len(samples) > 0:
        print(f"\nОценка {rating}:")
        for idx, row in samples.iterrows():
            text_preview = str(row['text'])[:60] + "..." if len(str(row['text'])) > 60 else str(row['text'])
            print(f"  - {text_preview}")

print("\n Схема преобразования:")
print("  0 (негатив) → НЕГАТИВНЫЙ (0)")
print("  1 (нейтральный) → НЕГАТИВНЫЙ (0)")
print("  2 (позитив) → ПОЗИТИВНЫЙ (1)")

df['label'] = df['rating'].apply(lambda x: 1 if x == 2 else 0)

print(f"\n Распределение после преобразования:")
neg_count = (df['label'] == 0).sum()
pos_count = (df['label'] == 1).sum()
total = len(df)

print(f"  НЕГАТИВНЫЙ (0,1 → 0): {neg_count} ({neg_count / total * 100:.1f}%)")
print(f"  ПОЗИТИВНЫЙ (2 → 1): {pos_count} ({pos_count / total * 100:.1f}%)")

print("\n Очистка текста...")


def clean_text(text):
    if not isinstance(text, str):
        text = str(text)

    text = text.lower()
    text = text.replace('ё', 'е')
    text = re.sub(r'[^а-яё\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text


df['clean_text'] = df['text'].apply(clean_text)
initial_len = len(df)
df = df[df['clean_text'].str.len() > 20]
print(f"  Удалено {initial_len - len(df)} слишком коротких отзывов")
print(f"  Осталось {len(df)} отзывов")

print("\n Подготовка данных для нейронной сети...")

texts = df['clean_text'].values
labels = df['label'].values

if len(texts) < 100:
    print(f"Мало данных: {len(texts)} отзывов. Увеличьте датасет.")
else:

    if pos_count < neg_count * 0.5 or pos_count > neg_count * 1.5:
        print(" Балансировка классов...")
        min_count = min(pos_count, neg_count)
        sample_size = min(10000, min_count)

        pos_df = df[df['label'] == 1].sample(sample_size, random_state=42)
        neg_df = df[df['label'] == 0].sample(sample_size, random_state=42)

        df_balanced = pd.concat([pos_df, neg_df]).sample(frac=1, random_state=42).reset_index(drop=True)

        texts = df_balanced['clean_text'].values
        labels = df_balanced['label'].values

        print(f"  После балансировки: {len(texts)} отзывов")
        print(f"    Позитивных: {len(df_balanced[df_balanced['label'] == 1])}")
        print(f"    Негативных: {len(df_balanced[df_balanced['label'] == 0])}")

train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

VOCAB_SIZE = 15000
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

train_lengths = [len(seq) for seq in train_sequences]
MAX_LENGTH = min(200, int(np.percentile(train_lengths, 95))) if train_lengths else 100

print(f"  Средняя длина текста: {np.mean(train_lengths):.1f} слов" if train_lengths else "  Нет данных")
print(f"  Используем MAX_LENGTH: {MAX_LENGTH} слов")

train_padded = pad_sequences(train_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')

print(f"\nРазмеры данных:")
print(f"  Обучающая выборка: {train_padded.shape}")
print(f"  Тестовая выборка: {test_padded.shape}")

print("\nСоздание RNN модели...")

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 128, input_length=MAX_LENGTH, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

print("\nОбучение модели...")
history = model.fit(
    train_padded, train_labels,
    epochs=10,
    batch_size=64,
    validation_split=0.1,
    verbose=1
)

print("\nОценка модели на тестовых данных...")
test_loss, test_acc = model.evaluate(test_padded, test_labels, verbose=0)
print(f"Точность на тесте: {test_acc:.2%}")

print("\nТестирование на примерах отзывов...")


def predict_sentiment(text):
    cleaned = clean_text(text)

    seq = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(seq, maxlen=MAX_LENGTH, padding='post', truncating='post')

    prediction = model.predict(padded, verbose=0)[0][0]

    if prediction > 0.5:
        return "ПОЗИТИВНЫЙ", prediction
    else:
        return "НЕГАТИВНЫЙ", 1 - prediction


test_samples = [
    "фильм просто супер очень понравилось рекомендую всем",
    "очень скучное кино не стоит потраченного времени",
    "актеры играли хорошо но сюжет слабоват",
    "один из лучших фильмов которые я видел обязательно к просмотру",
    "полный разочарование деньги на ветер",
    "великолепная режиссура и игра актеров на высоте",
    "ничего особенного средненько можно посмотреть если нечего делать",
    "шедеврально просто лучший фильм года без сомнений",
    "ужасная актерская игра и примитивный сюжет",
    "интересный замысел но плохая реализация"
]

print("=" * 60)
for i, text in enumerate(test_samples, 1):
    sentiment, confidence = predict_sentiment(text)
    print(f"{i:2d}. {sentiment} ({confidence:.1%})")
    print(f"    '{text[:50]}...'" if len(text) > 50 else f"    '{text}'")
print("=" * 60)

print("\n Точность по классам...")

from sklearn.metrics import classification_report

test_predictions = model.predict(test_padded, verbose=0)
test_pred_labels = (test_predictions > 0.5).astype(int).flatten()

print("\nОтчет о классификации:")
print(classification_report(test_labels, test_pred_labels,
                            target_names=['НЕГАТИВНЫЙ (0,1)', 'ПОЗИТИВНЫЙ (2)']))

print("\n Проверка на реальных отзывах из датасета...")

real_samples = df.sample(5, random_state=42)

print("\nРеальные отзывы из датасета:")
for i, (idx, row) in enumerate(real_samples.iterrows(), 1):
    original_rating = row['rating']
    original_label = "ПОЗИТИВНЫЙ" if row['rating'] == 2 else "НЕГАТИВНЫЙ"

    sentiment, confidence = predict_sentiment(row['text'])

    print(f"\n{i}. Исходная оценка: {original_rating} ({original_label})")
    print(f"   Предсказание: {sentiment} ({confidence:.1%})")
    text_preview = str(row['text'])[:70] + "..." if len(str(row['text'])) > 70 else str(row['text'])
    print(f"   Текст: {text_preview}")

    predicted_label = 1 if sentiment == " ПОЗИТИВНЫЙ" else 0
    actual_label = 1 if row['rating'] == 2 else 0
    if predicted_label == actual_label:
        print("ПРАВИЛЬНО")
    else:
        print("ОШИБКА")

 Загрузка данных...
 Загружено 210989 отзывов
Колонки: ['Unnamed: 0', 'text', 'sentiment']

 Первые 5 строк:
   Unnamed: 0                                               text  sentiment
0       43956  Развода на деньги нет\nНаблюдаюсь в Лайфклиник...          1
1       17755  Отель выбрали потому что рядом со стадионом. О...          0
2       20269  Вылечили\nГноился с рождения глазик, в поликли...          1
3       16648  Хорошее расположение.С вокзала дошли пешком.Но...          0
4       27879  Отличное месторасположение,прекрасный вид,особ...          1

 Преобразование рейтингов...
Уникальные значения оценок: [np.int64(0), np.int64(1), np.int64(2)]

 Примеры отзывов по оценкам:

Оценка 0:
  - Отель выбрали потому что рядом со стадионом. Отель 4*. Номер...
  - Хорошее расположение.С вокзала дошли пешком.Ночью очень тихо...

Оценка 1:
  - Развода на деньги нет
Наблюдаюсь в Лайфклиник по беременност...
  - Вылечили
Гноился с рождения глазик, в поликлинике назначили ...

Оценка 2:
  

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Обучение модели...
Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 24s 66ms/step - accuracy: 0.6275 - loss: 0.6244 - val_accuracy: 0.8106 - val_loss: 0.4137
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 14s 64ms/step - accuracy: 0.8845 - loss: 0.2966 - val_accuracy: 0.7975 - val_loss: 0.5323
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 14s 62ms/step - accuracy: 0.9682 - loss: 0.1101 - val_accuracy: 0.7819 - val_loss: 0.6353
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 14s 64ms/step - accuracy: 0.9933 - loss: 0.0295 - val_accuracy: 0.7763 - val_loss: 0.8501
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 14s 61ms/step - accuracy: 0.9944 - loss: 0.0194 - val_accuracy: 0.7869 - val_loss: 0.8959
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 21s 63ms/step - accuracy: 0.9952 - loss: 0.0162 - val_accuracy: 0.7825 - val_loss: 1.0043
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 14s 61ms/step - accuracy: 0.9938 - loss: 0.0192 - val_accuracy: 0.7844 - val_loss: 0.9576
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 14s 61ms/step - accuracy: 0.99